# Day 7: Handy Haversacks
https://adventofcode.com/2020/day/7

In [1]:
inputLines = lines <$> readFile "input/day07.txt"

In [2]:
import qualified Data.Map as Map

In [3]:
import Text.Regex.PCRE  -- install with 'stack install regex-pcre'
import Data.List.Split (splitOn)  -- install with 'stack install split'

In [4]:
testInput = 
  [ "light red bags contain 1 bright white bag, 2 muted yellow bags."
  , "dark orange bags contain 3 bright white bags, 4 muted yellow bags."
  , "bright white bags contain 1 shiny gold bag."
  , "muted yellow bags contain 2 shiny gold bags, 9 faded blue bags."
  , "shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags."
  , "dark olive bags contain 3 faded blue bags, 4 dotted black bags."
  , "vibrant plum bags contain 5 faded blue bags, 6 dotted black bags."
  , "faded blue bags contain no other bags."
  , "dotted black bags contain no other bags." ]

# Parse the given rules
The rules are represented by a nested map:
* The keys are the bag types
* The values are maps that describe the contents of each bag type:
    * The keys of the inner maps are the types of the contained bags
    * The values are the numbers of those bags that are contained in the outer bag

In [5]:
parseRules :: [String] -> Map.Map String (Map.Map String Int)
parseRules = 
    let
        parseRule :: String -> (String, Map.Map String Int)
        parseRule rule = (bagType, parseContents contents)
            where
                [_, bagType, contents] = head (rule =~ "^(.*) bags contain (.*)\\.$" :: [[String]])

        parseContents :: String -> Map.Map String Int
        parseContents "no other bags" = Map.empty
        parseContents contents = Map.fromList . map parseItem . splitOn ", " $ contents
        
        parseItem :: String -> (String, Int)
        parseItem item = (bagType, read number)
            where
                [_, number, bagType] = head (item =~ "^([0-9]+) (.*) bags?$" :: [[String]])
    in
        Map.fromList . map parseRule

# Part 1
`hasBag rules innerBag outerBag` is `True` if `outerBag` contains `innerBag` (either directly or indirectly) according to the given rules.

In [6]:
hasBag :: Map.Map String (Map.Map String Int) -> String -> String -> Bool
hasBag rules innerBag outerBag
    | innerBag == outerBag = True
    | otherwise = Just True == (
                any (hasBag rules innerBag) <$> 
                (Map.keys <$> Map.lookup outerBag rules))

Find the number of bags that contain a `shiny gold` bag.

In [7]:
solution1 :: Map.Map String (Map.Map String Int) -> Int
solution1 rules = length $ filter (hasBag rules targetBag) bagTypes
    where
        targetBag = "shiny gold"
        bagTypes = Map.keys $ Map.delete targetBag rules

Verify given example.

In [8]:
solution1 . parseRules $ testInput

4

## Solution, part 1

In [9]:
solution1 . parseRules <$> inputLines

115

# Part 2

In [10]:
import Data.Maybe (fromJust)

Find out how many bags are contained inside a `shiny gold` bag. Note that the `shiny gold` bag itself is not counted.

In [11]:
solution2 :: Map.Map String (Map.Map String Int) -> Int
solution2 rules = numberOfBags startBag - 1  -- do not count the startBag
    where
        startBag = "shiny gold"
        numberOfBags bagType = Map.foldlWithKey' accumulate 1 .
                               fromJust .
                               Map.lookup bagType $ rules
        accumulate current bagType number = current + 
                                            number * numberOfBags bagType

Verify given example.

In [12]:
solution2 . parseRules $ testInput

32

## Solution, part 2

In [13]:
solution2 . parseRules <$> inputLines

1250